In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# 载入 Excel 文件
file_path = 'PERM_Disclosure_Data_FY2023_Q4.xlsx'
xls = pd.ExcelFile(file_path)

# 查看工作表名称
print("工作表列表:", xls.sheet_names)

# 读取主数据表（假设为第一个）
df = xls.parse(xls.sheet_names[0])

# 显示列名以便后续匹配
print("列名示例：", df.columns.tolist())

In [ ]:
# 标准化字段（请根据你文件实际列名进行修改）
df['case_status'] = df['CASE_STATUS'].str.strip()
df['salary'] = pd.to_numeric(df['WAGE_OFFERED_FROM_9089'], errors='coerce')
df['job_title'] = df['JOB_TITLE']
df['employer_state'] = df['EMPLOYER_STATE']
df['education_level'] = df['EDUCATION_LEVEL']
df['is_certified'] = df['case_status'] == 'Certified'

# 处理时间字段
df['received_date'] = pd.to_datetime(df['RECEIVED_DATE'], errors='coerce')
df['decision_date'] = pd.to_datetime(df['DECISION_DATE'], errors='coerce')
df['processing_days'] = (df['decision_date'] - df['received_date']).dt.days

# 去除缺失值
df_clean = df.dropna(subset=['salary', 'job_title', 'employer_state', 'processing_days', 'education_level'])
sns.set(style="whitegrid")

In [ ]:
grouped = df_clean.groupby('job_title').agg(
    avg_salary=('salary', 'mean'),
    pass_rate=('is_certified', 'mean'),
    count=('salary', 'count')
).reset_index()

grouped = grouped[grouped['count'] > 50]

plt.figure(figsize=(10,6))
sns.scatterplot(data=grouped, x='avg_salary', y='pass_rate', size='count', hue='job_title', legend=False)
plt.title("职位平均薪资 vs 申请通过率")
plt.xlabel("平均薪资 ($)")
plt.ylabel("通过率")
plt.grid(True)
plt.show()

In [ ]:
state_proc = df_clean.groupby('employer_state')['processing_days'].mean().sort_values()

plt.figure(figsize=(10,6))
sns.barplot(x=state_proc.values, y=state_proc.index, palette='coolwarm')
plt.title("各州平均处理时间（天）")
plt.xlabel("平均处理时间")
plt.ylabel("州")
plt.grid(axis='x')
plt.show()

In [ ]:
edu_group = df_clean.groupby('education_level')['is_certified'].mean().sort_values(ascending=False)

plt.figure(figsize=(8,6))
sns.barplot(x=edu_group.index, y=edu_group.values, palette='viridis')
plt.title("不同学历申请者的通过率")
plt.xlabel("学历")
plt.ylabel("通过率")
plt.ylim(0.6, 1.0)
plt.grid(axis='y')
plt.xticks(rotation=30)
plt.show()